In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

In [16]:
# Data preparation
%run manageDataAmaury.ipynb
trainingSetValues = trainingSetValues
trainingSetLabels = trainingSetLabels
testSetValues = testSetValues

In [17]:
# import for the method
import pandas_profiling
import geopandas as gpd
from sklearn.model_selection import train_test_split

In [18]:
# analysis of data quality of the data set
# pandas_profiling.ProfileReport(trainingSetValues)

Pas besoin de supprimer des variables

In [19]:
# correlation maxtrix
cor = trainingSetValues.corr()
sns.heatmap(cor, square=True, cmap="coolwarm", annot=True)

<AxesSubplot:>

In [20]:
# transformation of id in index
pd.DataFrame.set_index(trainingSetLabels, 'id', inplace=True)
# trainingSetValues.drop(columns='population', inplace=True)
# testSetValues.drop(columns='population', inplace=True)

In [21]:
# encoding of categorical features
from sklearn.preprocessing import LabelEncoder

def encodeCategoricalFeature(dfTrain, dfTest):
    features = trainingSetValues.columns
    numericalFeatures = trainingSetValues._get_numeric_data().columns
    categoricalFeatures = list(set(features) - set(numericalFeatures))
    enc = LabelEncoder()
    for feature in categoricalFeatures:
        enc.fit(pd.concat([dfTest[feature], dfTrain[feature]]))
        dfTrain[feature] = enc.transform(dfTrain[feature])
        dfTest[feature] = enc.transform(dfTest[feature])

encodeCategoricalFeature(trainingSetValues, testSetValues)
#pandas_profiling.ProfileReport(trainingSetValues)

In [22]:
x_train, x_test, y_train, y_test = train_test_split(trainingSetValues, trainingSetLabels, test_size=0.25, random_state=66)

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer
rf = RandomForestClassifier(n_estimators=100, random_state=66)
y = np.ravel(trainingSetLabels)
x = trainingSetValues
models = cross_validate(rf, x, y, cv=10, scoring=make_scorer(accuracy_score), return_estimator=True)
argModel = np.argmax(models['test_score'])
model = models['estimator'][argModel]
print(models['test_score'][argModel])

0.815993265993266


In [24]:
predictions = model.predict(x_test)
score = accuracy_score(y_test, predictions)
print('Accuracy score on x_test: ' + str(score))

Accuracy score on x_test: 0.9804713804713805


In [27]:
# make prediction on new values
testPrediction = model.predict(testSetValues)
predictionDf = pd.DataFrame()
predictionDf['id'] = testSetValues.index
predictionDf['status_group'] = testPrediction
pd.DataFrame.set_index(predictionDf, 'id', inplace=True)
print(predictionDf)
predictionDf.to_csv('data/results.csv')

         status_group
id                   
50785  non functional
51630      functional
17168      functional
45559  non functional
49871      functional
...               ...
39307  non functional
18990      functional
28749      functional
33492      functional
68707  non functional

[14850 rows x 1 columns]
